In [1]:
import absl
import os
import tempfile
import time

import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tfx

from pprint import pprint
from tensorflow_metadata.proto.v0 import schema_pb2, statistics_pb2, anomalies_pb2
from tensorflow_transform.tf_metadata import schema_utils
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import InfraValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components import Tuner
from tfx.dsl.components.base import executor_spec
from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.components.trainer import executor as trainer_executor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import infra_validator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import HyperParameters
from tfx.types.standard_artifacts import ModelBlessing
from tfx.types.standard_artifacts import InfraBlessing

from tfx.utils import dsl_utils

In [2]:
ARTIFACT_STORE = os.path.join(os.sep, 'Users', 'CarlosMonsivais', 'Desktop', 'CPI Project', 'artifact-store')
SERVING_MODEL_DIR=os.path.join(os.sep, 'Users', 'CarlosMonsivais', 'Desktop', 'CPI Project', 'Model')
DATA_ROOT = '/Users/CarlosMonsivais/Desktop/CPI Project/Data'

In [3]:
PIPELINE_NAME = 'pipeline-trial1'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME, time.strftime("%Y%m%d_%H%M%S"))
os.makedirs(PIPELINE_ROOT, exist_ok=True)

In [4]:
context = InteractiveContext(pipeline_name = PIPELINE_NAME,
                             pipeline_root = PIPELINE_ROOT,
                             metadata_connection_config = None)

In [5]:
# from GCS_Classes.gcs_client import *
# from Variables.create_vars import *
# gcs_client_connect = GCS_Client_Connect()
# data_client = gcs_client_connect.create_client(project = gcp_project_id_bucket, bucket_name = gcp_bucket_name)

# ExampleGen

In [6]:
input = example_gen_pb2.Input(splits=[example_gen_pb2.Input.Split(name='train', pattern='*training*'),
                                      example_gen_pb2.Input.Split(name='eval', pattern='*testing*')])
example_gen = CsvExampleGen(input_base = DATA_ROOT, 
                            input_config = input)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/Users/CarlosMonsivais/Desktop/CPI Project/artifact-store/pipeline-trial1/20220414_231847/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:train,num_files:1,total_bytes:33468,xor_checksum:1649981388,sum_checksum:1649981388\nsplit:eval,num_files:1,total_bytes:8658,xor_checksum:1649981387,sum_checksum:1649981387"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

# StatisticsGen

In [7]:
statistics_gen = tfx.components.StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

2022-04-14 23:18:54.183905: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/Users/CarlosMonsivais/Desktop/CPI Project/artifact-store/pipeline-trial1/20220414_231847/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [8]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


# SchemaGen

In [9]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'],
                       infer_feature_shape=False)
context.run(schema_gen)

2022-04-14 23:18:57.155558: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/Users/CarlosMonsivais/Desktop/CPI Project/artifact-store/pipeline-trial1/20220414_231847/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

In [10]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'date',BYTES,required,single,-
'item',STRING,required,single,'item'
'series_id',STRING,required,single,'series_id'
'value',FLOAT,required,single,-


/Users/CarlosMonsivais/Desktop/CPI Project/cpi_env/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'item','Airline fares'
'series_id','CUUR0000SETG01 '


# ExampleValidator

In [11]:
example_validator = ExampleValidator(instance_name = "Data_Validation",    
                                     statistics = statistics_gen.outputs['statistics'],
                                     schema = schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator.Data_Validation
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/Users/CarlosMonsivais/Desktop/CPI Project/artifact-store/pipeline-trial1/20220414_231847/ExampleValidator.Data_Validation/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator.Data_Validation"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [12]:
context.show(example_validator.outputs['output'])

# Trainer Component

In [19]:
TRAINER_MODULE_FILE = '/Users/CarlosMonsivais/Desktop/CPI Project/Model/tfx_model.py'

In [20]:
trainer = Trainer(module_file=TRAINER_MODULE_FILE,
                  examples=example_gen.outputs['examples'],
                  schema = schema_gen.outputs['schema'],
                  train_args = trainer_pb2.TrainArgs(splits=['train']),
                  eval_args = trainer_pb2.EvalArgs(splits=['eval']))

In [21]:
context.run(trainer)

AttributeError: module 'user_module' has no attribute 'trainer_fn'